<a href="https://colab.research.google.com/github/11doris/jazz-maestro/blob/colab_word_embeddings/section_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall gensim -y

Found existing installation: gensim 3.6.0
Uninstalling gensim-3.6.0:
  Successfully uninstalled gensim-3.6.0


In [2]:
!pip install gensim

     |████████████████████████████████| 24.1 MB 1.5 kB/s 


In [3]:
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pprint
import pandas as pd
import numpy as np
from collections import Counter
import plotly.express as px
from tqdm import tqdm 
from gensim.models.doc2vec import Doc2Vec
import pickle
import os

In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
print(gensim.__version__)

4.1.2


# Initialization

## Download the Data

### Download Data

In [6]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO' -O data.csv

--2021-11-17 14:08:47--  https://docs.google.com/uc?export=download&id=1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO
Resolving docs.google.com (docs.google.com)... 74.125.70.101, 74.125.70.113, 74.125.70.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.70.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/puqpl4b71ecubqoirqu46onoa3vkago0/1637158125000/14329102864480165501/*/1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO?e=download [following]
--2021-11-17 14:08:48--  https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/puqpl4b71ecubqoirqu46onoa3vkago0/1637158125000/14329102864480165501/*/1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO?e=download
Resolving doc-0g-4c-docs.googleusercontent.com (doc-0g-4c-docs.googleusercontent.com)... 108.177.120.132, 2607:f8b0:4001:c18::84
Connecting to doc-0g-4c-docs.googleusercontent.com (doc-0g-4c-docs

In [7]:
path_to_file = '/content/data.csv'
df = pd.read_csv(path_to_file, sep='\t', index_col="id")
df = df.reset_index()

In [8]:
df

,id,file_name,title,tune_mode,tune_id,section_name,section_id,chords
0,0,dataset/jazz1350/26-2.xml,26-2,major,0,A,1,C Eb7 G# B7 E G7 Gm C7 F G#7 C# E7 Am D7 Dm G7
1,1,dataset/jazz1350/26-2.xml,26-2,major,0,A,2,C Eb7 G# B7 E G7 Gm C7 F Eb7 G# B7 E G7 C
2,2,dataset/jazz1350/26-2.xml,26-2,major,0,B,3,Gm C7 Bm E7 A C7 F Bbm Eb7 G# Dm G7
3,3,dataset/jazz1350/26-2.xml,26-2,major,0,A,4,C Eb7 G# B7 E G7 Gm C7 F Eb7 G# B7 E G7 C
4,4,dataset/jazz1350/500 Miles High.xml,500 Miles High,minor,1,NaN,0,Am Am Cm Cm Eb Eb Em A7 Dm Dm Bm Bm Bbm Bbm Fm...
...,...,...,...,...,...,...,...,...
5398,5398,dataset/trad/You're Lucky To Me.xml,You're Lucky To Me,major,1808,A,1,Dm G7 Em A7 Dm G#7 G7 C C
5399,5399,dataset/trad/You're Lucky To Me.xml,You're Lucky To Me,major,1808,A,2,Dm G7 Em A7 Dm G#7 G7 C C A7 A7 A7 A7 Dm G7 D7...
5400,5400,dataset/trad/You're Nobody Till Somebody Loves...,You're Nobody Till Somebody Loves You,major,1809,A,1,C E7 A7 A7 Dm G7 C Em C G7 Em Cm Dm Dm Dm D7 D...
5401,5401,dataset/trad/Your Feet's Too Big.xml,Your Feet's Too Big,major,1810,NaN,0,G7 G7 C C G7 G7 C F G# C C G7 G7 C F C C G7 C ...


### Meta Data

In [9]:
titles = df.loc[:, ['id', 'tune_id', 'section_id', 'section_name', 'title', 'tune_mode']]

In [10]:
titles

,id,tune_id,section_id,section_name,title,tune_mode
0,0,0,1,A,26-2,major
1,1,0,2,A,26-2,major
2,2,0,3,B,26-2,major
3,3,0,4,A,26-2,major
4,4,1,0,NaN,500 Miles High,minor
...,...,...,...,...,...,...
5398,5398,1808,1,A,You're Lucky To Me,major
5399,5399,1808,2,A,You're Lucky To Me,major
5400,5400,1809,1,A,You're Nobody Till Somebody Loves You,major
5401,5401,1810,0,NaN,Your Feet's Too Big,major


In [11]:
##TODO Now a tune contains multiple rows

In [12]:
titles_dict = titles.to_dict()
sectionid_to_title = titles_dict['title']
sectionid_to_title[1170]

'Emily'

In [13]:
sectionid_to_title_id = titles_dict['tune_id']
sectionid_to_title_id[5396]

1807

In [14]:
titles_rows = titles.to_dict(orient='records')
titles_rows[0]

sectionid_to_section = []
for i, row in enumerate(titles_rows):
  name = f"{row['title']}, section{row['section_id']} ({row['section_name']})"
  sectionid_to_section.append(name)

sectionid_to_section[3]

'26-2, section4 (A)'

In [15]:
title_to_sectionid = {}

for row in titles.iterrows():
  title = row[1]['title']
  if title not in title_to_sectionid:
    title_to_sectionid[title] = [row[1]['id']]
  else:
    title_to_sectionid[title].append(row[1]['id'])


In [16]:
title_to_sectionid

{'26-2': [0, 1, 2, 3],
 '500 Miles High': [4],
 '502 Blues': [5],
 '52nd Street Theme': [6, 7, 8, 9],
 '9.20 Special': [10, 11, 12, 13],
 'A Ballad': [14, 15, 16, 17],
 'A Beautiful Friendship': [18, 19, 20, 21],
 'A Blossom Fell': [22, 23],
 'A Certain Smile': [24, 25, 26, 27],
 'A Child Is Born': [28, 29],
 'A Felicidade': [30, 31, 32, 33, 34],
 'A Fine Romance': [35, 36, 37, 38],
 'A Flower Is A Lovesome Thing': [39, 40, 41, 42],
 'A Foggy Day': [43, 44, 45, 46],
 'A Ghost Of A Chance': [47, 48, 49, 50],
 'A Kiss To Build A Dream On': [51, 52, 53, 54],
 'A Little Tear': [55, 56],
 'A Lovely Way To Spend An Evening': [57, 58, 59, 60],
 'A Night In Tunisia': [61, 62, 63, 64],
 'A Nightingale Sang In Berkeley Square': [65, 66, 67, 68],
 'A Pretty Girl Is Like A Melody': [69, 70],
 "A Sleepin' Bee": [71, 72, 73, 74],
 'A Smooth One': [75, 76, 77, 78],
 'A Sound For Sore Ears': [79, 80, 81],
 'A Sunday Kind Of Love': [82, 83, 84, 85],
 'A Taste Of Honey': [86, 87],
 'A Time For Love': [8

# List Tunes for Test Data

In [20]:
contrafacts = {
               "26-2": "Confirmation",
               "52nd Street Theme": "I Got Rhythm",
               "Ablution": "All The Things You Are",
               "Anthropology": "I Got Rhythm",
               "Ballade": "As Long As I Live",
               "Bright Mississippi": "Sweet Georgia Brown",
               "C.T.A.": "I Got Rhythm",
               "Celia": "I Got Rhythm",
               "Cottontail": "I Got Rhythm",
               "Countdown": "Tune Up",
               "Dewey Square": "Oh, Lady Be Good",
               "Dexterity": "I Got Rhythm",
               "Dig": "Sweet Georgia Brown",
               "Donna Lee": "Indiana (Back Home Again In)",
               "Don't Be That Way": "I Got Rhythm",
               "Eternal Triangle": "I Got Rhythm",
               "Evidence": "Just You, Just Me",
               "Flintstones": "I Got Rhythm",
               "Four On Six": "Summertime",
               "Freight Train": "Blues For Alice",
               "Good Bait": "I Got Rhythm",
               "Hackensack": "Oh, Lady Be Good",
               "Half Nelson": "Lady Bird",
               "Hot House": "What Is This Thing Called Love",
               "Impressions": "So What",
               "In A Mellow Tone (In A Mellotone)": "Rose Room",
               "In Walked Bud": "Blue Skies",
               "Ko Ko": "Cherokee",
               "Lennie's Pennies": "Pennies From Heaven",   ## Lennie's Pennies is in minor and therefore transposed to Amin... not possible to recognize like that
               # "Let's Call This": "Honeysuckle Rose",
               "Little Rootie Tootie": "I Got Rhythm",
               "Little Willie Leaps": "All God's Chillun Got Rhythm",
               "Lullaby Of Birdland": "Love Me Or Leave Me",
               "Moose The Mooche": "I Got Rhythm",
               "My Little Suede Shoes": "Jeepers Creepers",
               "Oleo": "I Got Rhythm",
               "Ornithology": "How High The Moon",
               "Passport": "I Got Rhythm",
               "Quasimodo (Theme)": "Embraceable You",
               "Rhythm-a-ning": "I Got Rhythm",
               "Room 608": "I Got Rhythm",
               "Salt Peanuts": "I Got Rhythm",
               "Satellite": "How High The Moon",
               "Scrapple From The Apple": "Honeysuckle Rose", # A section
               "Scrapple From The Apple": "I Got Rhythm", # B section
               #"Segment": "I Got Rhythm",
               "Seven Come Eleven": "I Got Rhythm",
               "Shaw 'Nuff": "I Got Rhythm",
               "Theme, The": "I Got Rhythm",
               "Tour De Force": "Jeepers Creepers",
               "Wow": "You Can Depend On Me",
               "Yardbird Suite": "Rosetta",
               # following tunes are not from wikipedia:
               "Sweet Sue, Just You": "Honeysuckle Rose",  # A section
               # "All Of Me": "Pennies From Heaven", # bars 25-28 of All of Me are same as bars 17-20 of Pennies From Heaven, but different key!
               "Sweet Sue, Just You": "Bye Bye Blackbird", # Bridge same
               "These Foolish Things": "Blue Moon", # first 8 bars same
               
}

contrafacts_test = []
for reference, compare in contrafacts.items():
  contrafacts_test.append([title_to_sectionid[reference], title_to_sectionid[compare]])
  

In [21]:
contrafacts_test[:5]

[[[0, 1, 2, 3], [810, 811, 812, 813]],
 [[6, 7, 8, 9], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]],
 [[104, 105, 106, 107], [209, 210, 211, 212]],
 [[272, 273, 274, 275], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]],
 [[335, 336], [297, 298, 299, 300, 4475, 4476, 4477]]]

## Helpers for N-Grams

In [22]:
def ngrams(tokens, n=2, sep='-'):
    return [sep.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])]

In [23]:
def list_to_ngrams(data, n=2, sep='-'):
  out = []
  for row in data:
    out.append(ngrams(row, n, sep))

  return out

# TF-IDF

## Build Model

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
lines = df.loc[:, 'chords'].tolist()
data = [line.split(' ') for line in lines]

In [26]:
tfidf_ngrams = [1, 2, 3, 4]

tfidf_model = {}
for n in tfidf_ngrams:

  data_ngrams = list_to_ngrams(data, n=n)
  print(data_ngrams[0])
  data_tfidf = [" ".join(i) for i in data_ngrams]

  tfidf_vectorizer = TfidfVectorizer()
  tfidf_model[n] = tfidf_vectorizer.fit_transform(data_tfidf)

['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am', 'D7', 'Dm', 'G7']
['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm', 'Gm-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am', 'Am-D7', 'D7-Dm', 'Dm-G7']
['C-Eb7-G#', 'Eb7-G#-B7', 'G#-B7-E', 'B7-E-G7', 'E-G7-Gm', 'G7-Gm-C7', 'Gm-C7-F', 'C7-F-G#7', 'F-G#7-C#', 'G#7-C#-E7', 'C#-E7-Am', 'E7-Am-D7', 'Am-D7-Dm', 'D7-Dm-G7']
['C-Eb7-G#-B7', 'Eb7-G#-B7-E', 'G#-B7-E-G7', 'B7-E-G7-Gm', 'E-G7-Gm-C7', 'G7-Gm-C7-F', 'Gm-C7-F-G#7', 'C7-F-G#7-C#', 'F-G#7-C#-E7', 'G#7-C#-E7-Am', 'C#-E7-Am-D7', 'E7-Am-D7-Dm', 'Am-D7-Dm-G7']


In [27]:
tfidf_model

{1: <5403x20 sparse matrix of type '<class 'numpy.float64'>'
 	with 29720 stored elements in Compressed Sparse Row format>,
 2: <5403x20 sparse matrix of type '<class 'numpy.float64'>'
 	with 29720 stored elements in Compressed Sparse Row format>,
 3: <5403x20 sparse matrix of type '<class 'numpy.float64'>'
 	with 29717 stored elements in Compressed Sparse Row format>,
 4: <5403x20 sparse matrix of type '<class 'numpy.float64'>'
 	with 29716 stored elements in Compressed Sparse Row format>}

## Test for single tunes

In [28]:
tunes_eval_list = [
                   'These Foolish Things',

]

Evaluate TF-IDF Model for ngram=1 and ngram=2

In [39]:
model1 = 1
model2 = 2

df_sim = pd.DataFrame(columns=['sid1', 'sid2', 'section1', 'section2', f'sim{model1}', f'sim{model2}'])
for tune in tunes_eval_list:
  print(tune)
  for s1 in title_to_sectionid[tune]:
      print(s1)
      for s2 in range(len(data_tfidf)):
        sim1 = cosine_similarity(tfidf_model[model1][s1], tfidf_model[model1][s2])[0][0]
        sim2 = cosine_similarity(tfidf_model[model2][s1], tfidf_model[model2][s2])[0][0]
        if sim1 > 0.6:
          df_sim.loc[len(df_sim)] = [s1, s2, sectionid_to_section[s1], sectionid_to_section[s2], sim1, sim2]

df_sim = df_sim.sort_values(f'sim{model2}', ascending=False)

These Foolish Things
3866
3867
3868
3869


In [40]:
df_sim.head(50)

,sid1,sid2,section1,section2,sim1,sim2
5541,3869,3869,"These Foolish Things, section4 (A)","These Foolish Things, section4 (A)",1.000000,1.000000
3997,3868,3868,"These Foolish Things, section3 (B)","These Foolish Things, section3 (B)",1.000000,1.000000
1325,3866,3866,"These Foolish Things, section1 (A)","These Foolish Things, section1 (A)",1.000000,1.000000
2918,3867,3867,"These Foolish Things, section2 (A)","These Foolish Things, section2 (A)",1.000000,1.000000
5539,3869,3866,"These Foolish Things, section4 (A)","These Foolish Things, section1 (A)",0.992204,0.991027
1327,3866,3869,"These Foolish Things, section1 (A)","These Foolish Things, section4 (A)",0.992204,0.991027
3829,3868,2760,"These Foolish Things, section3 (B)","More Than You Know, section3 (B)",0.989604,0.987022
2919,3867,3869,"These Foolish Things, section2 (A)","These Foolish Things, section4 (A)",0.926155,0.972433
5540,3869,3867,"These Foolish Things, section4 (A)","These Foolish Things, section2 (A)",0.926155,0.972433
1326,3866,3867,"These Foolish Things, section1 (A)","These Foolish Things, section2 (A)",0.927843,0.965897


## Test of the Contrafacts tunes

In [41]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2']
for c in tfidf_model.keys():
  cols.append(f"cosval_ngram{c}")
df_eval_tfidf = pd.DataFrame(columns=cols)


In [42]:
for tune1, tune2 in contrafacts_test:
  print(f"{tune1}, {tune2}")

[0, 1, 2, 3], [810, 811, 812, 813]
[6, 7, 8, 9], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[104, 105, 106, 107], [209, 210, 211, 212]
[272, 273, 274, 275], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[335, 336], [297, 298, 299, 300, 4475, 4476, 4477]
[594], [3746, 3747]
[668, 669, 670, 671], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[707, 708, 709, 710, 711, 712, 713, 714], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[835, 836, 837, 838], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[841], [4028]
[980, 981, 982, 983], [2968, 2969, 2970, 2971]
[984, 985, 986, 987], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[995, 996], [3746, 3747]
[1084, 1085], [1996, 1997]
[1042, 1043, 1044, 1045], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[1191, 1192, 1193, 1194], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[1216, 1217, 1218, 1219], [2210, 2211, 2212, 2213]
[1271, 1272, 1273, 1274], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[1321], [3722, 5222]
[13

In [51]:
# calculate a dataframe with the cos sim value for the test tunes
df_tfidf = pd.DataFrame(columns=['tune1', 'tune2', 'ngram_size', 'section1', 'section2', 'similarity'])
for ngram_size, tfidf_m in tfidf_model.items():

    for tune1, tune2 in contrafacts_test:
      #print(f"tune1: {tune1}, tune2: {tune2}")
      tune_sim = []
      # loop over all combinations of sections
      for s1 in tune1:
          for s2 in tune2:
                #print(f"ngram: {ngram_size}: {sectionid_to_section[s1]}, {sectionid_to_section[s2]}:  {cosine_similarity(tfidf_m[s1], tfidf_m[s2])[0][0]}")
                row = [sectionid_to_title_id[tune1[0]], sectionid_to_title_id[tune2[0]], ngram_size, sectionid_to_section[s1], sectionid_to_section[s2], cosine_similarity(tfidf_m[s1], tfidf_m[s2])[0][0]]
                df_tfidf.loc[len(df_tfidf)] = row
      


In [52]:
df_tfidf.query('ngram_size == 1').sort_values(['tune1', 'similarity'], ascending=[True, False]).head(50)

,tune1,tune2,ngram_size,section1,section2,similarity
10,0,251,1,"26-2, section3 (B)","Confirmation, section3 (B)",0.875331
2,0,251,1,"26-2, section1 (A)","Confirmation, section3 (B)",0.604696
3,0,251,1,"26-2, section1 (A)","Confirmation, section4 (A)",0.601272
6,0,251,1,"26-2, section2 (A)","Confirmation, section3 (B)",0.596132
14,0,251,1,"26-2, section4 (A)","Confirmation, section3 (B)",0.596132
1,0,251,1,"26-2, section1 (A)","Confirmation, section2 (A)",0.567644
0,0,251,1,"26-2, section1 (A)","Confirmation, section1 (A)",0.552823
11,0,251,1,"26-2, section3 (B)","Confirmation, section4 (A)",0.546667
9,0,251,1,"26-2, section3 (B)","Confirmation, section2 (A)",0.545618
8,0,251,1,"26-2, section3 (B)","Confirmation, section1 (A)",0.468002


In [53]:
df_tfidf_pivot = pd.pivot_table(df_tfidf, index = ['tune1', 'tune2', 'section1', 'section2'], columns = 'ngram_size', values = 'similarity')
df_tfidf_pivot.head(50)

ngram_size                                                                               1  ...         4
tune1 tune2 section1                        section2                                        ...          
0     251   26-2, section1 (A)              Confirmation, section1 (A)            0.552823  ...  0.609988
                                            Confirmation, section2 (A)            0.567644  ...  0.620385
                                            Confirmation, section3 (B)            0.604696  ...  0.333160
                                            Confirmation, section4 (A)            0.601272  ...  0.636676
            26-2, section2 (A)              Confirmation, section1 (A)            0.193218  ...  0.220299
                                            Confirmation, section2 (A)            0.211227  ...  0.236902
                                            Confirmation, section3 (B)            0.596132  ...  0.479635
                                            Confirmation, section4 (A)            0.244534  ...  0.252052
            26-2, section3 (B)              Confirmation, section1 (A)            0.468002  ...  0.329679
                                            Confirmation, section2 (A)            0.545618  ...  0.357553
                                            Confirmation, section3 (B)            0.875331  ...  0.855653
                                            Confirmation, section4 (A)            0.546667  ...  0.358395
            26-2, section4 (A)              Confirmation, section1 (A)            0.193218  ...  0.220299
                                            Confirmation, section2 (A)            0.211227  ...  0.236902
                                            Confirmation, section3 (B)            0.596132  ...  0.479635
                                            Confirmation, section4 (A)            0.244534  ...  0.252052
3     505   52nd Street Theme, section1 (A) I Got Rhythm, section1 (A)            0.773683  ...  0.695144
                                            I Got Rhythm, section2 (A)            0.746507  ...  0.672647
                                            I Got Rhythm, section3 (B)            0.200034  ...  0.110334
                                            I Got Rhythm, section4 (A)            0.699371  ...  0.636858
            52nd Street Theme, section2 (A) I Got Rhythm, section1 (A)            0.773683  ...  0.695144
                                            I Got Rhythm, section2 (A)            0.746507  ...  0.672647
                                            I Got Rhythm, section3 (B)            0.200034  ...  0.110334
                                            I Got Rhythm, section4 (A)            0.699371  ...  0.636858
            52nd Street Theme, section3 (B) I Got Rhythm, section1 (A)            0.551293  ...  0.343902
                                            I Got Rhythm, section2 (A)            0.534023  ...  0.336950
                                            I Got Rhythm, section3 (B)            0.478726  ...  0.657322
                                            I Got Rhythm, section4 (A)            0.454257  ...  0.281391
            52nd Street Theme, section4 (A) I Got Rhythm, section1 (A)            0.773683  ...  0.695144
                                            I Got Rhythm, section2 (A)            0.746507  ...  0.672647
                                            I Got Rhythm, section3 (B)            0.200034  ...  0.110334
                                            I Got Rhythm, section4 (A)            0.699371  ...  0.636858
30    63    Ablution, section1 (A)          All The Things You Are, section1 (A)  1.000000  ...  1.000000
                                            All The Things You Are, section2 (B)  0.268451  ...  0.097962
                                            All The Things You Are, section3 (C)  0.000000  ...  0.000000
                                            All The Things You Are, section4 (

In [54]:
df_tfidf_pivot.query('tune1 == 1170')

ngram_size                                                                     1  ...         4
tune1 tune2 section1                           section2                           ...          
1170  159   These Foolish Things, section1 (A) Blue Moon, section1 (A)  0.796798  ...  0.720147
                                               Blue Moon, section2 (A)  0.684029  ...  0.657114
                                               Blue Moon, section3 (B)  0.620795  ...  0.458182
                                               Blue Moon, section4 (A)  0.796798  ...  0.720147
            These Foolish Things, section2 (A) Blue Moon, section1 (A)  0.729480  ...  0.718861
                                               Blue Moon, section2 (A)  0.634617  ...  0.665911
                                               Blue Moon, section3 (B)  0.542454  ...  0.459531
                                               Blue Moon, section4 (A)  0.729480  ...  0.718861
            These Foolish Things, section3 (B) Blue Moon, section1 (A)  0.406638  ...  0.365330
                                               Blue Moon, section2 (A)  0.551017  ...  0.436748
                                               Blue Moon, section3 (B)  0.372455  ...  0.292393
                                               Blue Moon, section4 (A)  0.406638  ...  0.365330
            These Foolish Things, section4 (A) Blue Moon, section1 (A)  0.816650  ...  0.743957
                                               Blue Moon, section2 (A)  0.700277  ...  0.688502
                                               Blue Moon, section3 (B)  0.639905  ...  0.482350
                                               Blue Moon, section4 (A)  0.816650  ...  0.743957

[16 rows x 4 columns]

In [55]:
df_tfidf_pivot.query('tune1 == 1138')

ngram_size                                                                            1  ...         4
tune1 tune2 section1                          section2                                   ...          
1138  206   Sweet Sue, Just You, section1 (A) Bye Bye Blackbird, section1 (A)  0.920229  ...  0.841311
                                              Bye Bye Blackbird, section2 (B)  0.633623  ...  0.606482
                                              Bye Bye Blackbird, section3 (C)  0.506609  ...  0.505658
            Sweet Sue, Just You, section2 (A) Bye Bye Blackbird, section1 (A)  0.920229  ...  0.841311
                                              Bye Bye Blackbird, section2 (B)  0.633623  ...  0.606482
                                              Bye Bye Blackbird, section3 (C)  0.506609  ...  0.505658
            Sweet Sue, Just You, section3 (B) Bye Bye Blackbird, section1 (A)  0.396243  ...  0.572275
                                              Bye Bye Blackbird, section2 (B)  0.547218  ...  0.761143
                                              Bye Bye Blackbird, section3 (C)  0.554769  ...  0.795095
            Sweet Sue, Just You, section4 (A) Bye Bye Blackbird, section1 (A)  0.920229  ...  0.841311
                                              Bye Bye Blackbird, section2 (B)  0.633623  ...  0.606482
                                              Bye Bye Blackbird, section3 (C)  0.506609  ...  0.505658

[12 rows x 4 columns]

# Doc2Vec Embeddings

## Read and Tag Data

In [56]:
df.head()

,id,file_name,title,tune_mode,tune_id,section_name,section_id,chords
0,0,dataset/jazz1350/26-2.xml,26-2,major,0,A,1,C Eb7 G# B7 E G7 Gm C7 F G#7 C# E7 Am D7 Dm G7
1,1,dataset/jazz1350/26-2.xml,26-2,major,0,A,2,C Eb7 G# B7 E G7 Gm C7 F Eb7 G# B7 E G7 C
2,2,dataset/jazz1350/26-2.xml,26-2,major,0,B,3,Gm C7 Bm E7 A C7 F Bbm Eb7 G# Dm G7
3,3,dataset/jazz1350/26-2.xml,26-2,major,0,A,4,C Eb7 G# B7 E G7 Gm C7 F Eb7 G# B7 E G7 C
4,4,dataset/jazz1350/500 Miles High.xml,500 Miles High,minor,1,NaN,0,Am Am Cm Cm Eb Eb Em A7 Dm Dm Bm Bm Bbm Bbm Fm...


In [58]:
def read_corpus(df, ngram_size, tokens_only=False):
    for i, row in df.iterrows():
      tokens = ngrams(row['chords'].split(), ngram_size, '-')
      if tokens_only:
          yield tokens
      else:
          # for training data, add tags
          yield gensim.models.doc2vec.TaggedDocument(tokens, [row['id'], row['tune_id']])
          #yield gensim.models.doc2vec.TaggedDocument(tokens, [row['id']])

## Doc2Vec Hyperparameters

In [59]:
!rm -R models_doc2vec
!rm -R traindata_doc2vec

rm: cannot remove 'models_doc2vec': No such file or directory
rm: cannot remove 'traindata_doc2vec': No such file or directory


In [60]:
!mkdir models_doc2vec
!mkdir traindata_doc2vec

In [63]:
import multiprocessing

model_path = './models_doc2vec'
model_prefix = 'autos'
data_path = './traindata_doc2vec'

param_grid = {'d2v': {'variant': ['pv-dm', 
                                  'concat',
                                  'pv-dbow'
                                  ], 
                      'window': [
                                 2, 
                                 3, 
                                 4, 
                                 10
                                 ],
                      'ngrams': [
                                 1, 
                                 2, 
                                 3, 
                                 4
                                 ],
                     }
              }


common_kwargs = dict(
    vector_size=100, 
    epochs=40, 
    min_count=1,
    sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=10, 
    hs=0,
)

for algo, params in param_grid.items():
    for variant in params['variant']:
        dm = 0 if variant == 'pv-dbow' else 1
        concat = 1 if variant ==  'concat' else 0

        for ngram_size in params['ngrams']:
            
            # prepare data with the provided ngram size
            train_corpus = list(read_corpus(df, ngram_size=ngram_size))

            # evaluate different window sizes
            for window in params['window']:

                # calculate the model
                if algo == 'd2v':
                    print(f"Model: {variant}, dm={dm}, dm_concat={concat}; ngrams={ngram_size}, window={window}")
                    print(f"Corpus: {train_corpus[0]}")
                    model = gensim.models.doc2vec.Doc2Vec(**common_kwargs,
                                                          dm = dm,
                                                          dm_concat = concat,
                                                          )
                    model.build_vocab(train_corpus)
                    model.train(train_corpus, 
                                total_examples=model.corpus_count, 
                                epochs=model.epochs)

                else:
                    model = gensim.models.FastText(data_ngrams, vector_size=size, window=window, sg=sg, epochs=50)

                file_name = f"{model_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}"
                model.save(file_name + '.model')

                file_name = f"{data_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}"
                with open(file_name + '.train', 'wb') as f:
                      # Pickle the 'data' dictionary using the highest protocol available.
                      pickle.dump(train_corpus, f, pickle.HIGHEST_PROTOCOL)

2021-11-17 14:25:32,491 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n10,w5,t2)', 'datetime': '2021-11-17T14:25:32.491331', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-17 14:25:32,493 : INFO : collecting all words and their counts
2021-11-17 14:25:32,497 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-11-17 14:25:32,526 : INFO : collected 36 word types and 5403 unique tags from a corpus of 5403 examples and 82790 words
2021-11-17 14:25:32,528 : INFO : Creating a fresh vocabulary
2021-11-17 14:25:32,533 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 retains 36 unique words (100.0%% of original 36, drops 0)', 'datetime': '2021-11-17T14:25:32.533089', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-

Model: pv-dm, dm=1, dm_concat=0; ngrams=1, window=2
Corpus: TaggedDocument(['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am', 'D7', 'Dm', 'G7'], [0, 0])


2021-11-17 14:25:33,227 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-17 14:25:33,234 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-17 14:25:33,236 : INFO : EPOCH - 1 : training on 82790 raw words (93596 effective words) took 0.6s, 144301 effective words/s
2021-11-17 14:25:33,871 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-17 14:25:33,891 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-17 14:25:33,893 : INFO : EPOCH - 2 : training on 82790 raw words (93596 effective words) took 0.6s, 145049 effective words/s
2021-11-17 14:25:34,471 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-17 14:25:34,488 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-17 14:25:34,490 : INFO : EPOCH - 3 : training on 82790 raw words (93596 effective words) took 0.6s, 160364 effective words/s
2021-11-17 14:25:35,088 : INFO : worker thread finished

Model: pv-dm, dm=1, dm_concat=0; ngrams=1, window=3
Corpus: TaggedDocument(['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am', 'D7', 'Dm', 'G7'], [0, 0])


2021-11-17 14:26:01,558 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-17 14:26:01,573 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-17 14:26:01,574 : INFO : EPOCH - 1 : training on 82790 raw words (93596 effective words) took 0.6s, 160803 effective words/s
2021-11-17 14:26:02,117 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-17 14:26:02,137 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-17 14:26:02,138 : INFO : EPOCH - 2 : training on 82790 raw words (93596 effective words) took 0.6s, 168283 effective words/s
2021-11-17 14:26:02,699 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-17 14:26:02,715 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-17 14:26:02,716 : INFO : EPOCH - 3 : training on 82790 raw words (93596 effective words) took 0.6s, 165749 effective words/s
2021-11-17 14:26:03,284 : INFO : worker thread finished

...

2021-11-17 14:37:23,877 : INFO : deleting the raw counts dictionary of 14361 items
2021-11-17 14:37:23,879 : INFO : sample=0 downsamples 0 most-common words
2021-11-17 14:37:23,888 : INFO : Doc2Vec lifecycle event {'msg': 'downsampling leaves estimated 66584 word corpus (100.0%% of prior 66584)', 'datetime': '2021-11-17T14:37:23.887955', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2021-11-17 14:37:24,086 : INFO : estimated required memory for 14361 words and 100 dimensions: 21911100 bytes
2021-11-17 14:37:24,087 : INFO : resetting layer weights
2021-11-17 14:37:24,106 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 2 workers on 14361 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=10 window=5 shrink_windows=True', 'datetime': '2021-11-17T14:37:24.106592', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GC

Model: pv-dbow, dm=0, dm_concat=0; ngrams=4, window=4
Corpus: TaggedDocument(['C-Eb7-G#-B7', 'Eb7-G#-B7-E', 'G#-B7-E-G7', 'B7-E-G7-Gm', 'E-G7-Gm-C7', 'G7-Gm-C7-F', 'Gm-C7-F-G#7', 'C7-F-G#7-C#', 'F-G#7-C#-E7', 'G#7-C#-E7-Am', 'C#-E7-Am-D7', 'E7-Am-D7-Dm', 'Am-D7-Dm-G7'], [0, 0])


2021-11-17 14:37:44,732 : INFO : deleting the raw counts dictionary of 14361 items
2021-11-17 14:37:44,734 : INFO : sample=0 downsamples 0 most-common words
2021-11-17 14:37:44,735 : INFO : Doc2Vec lifecycle event {'msg': 'downsampling leaves estimated 66584 word corpus (100.0%% of prior 66584)', 'datetime': '2021-11-17T14:37:44.735676', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2021-11-17 14:37:44,927 : INFO : estimated required memory for 14361 words and 100 dimensions: 21911100 bytes
2021-11-17 14:37:44,929 : INFO : resetting layer weights
2021-11-17 14:37:44,948 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 2 workers on 14361 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=10 window=5 shrink_windows=True', 'datetime': '2021-11-17T14:37:44.948049', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GC

Model: pv-dbow, dm=0, dm_concat=0; ngrams=4, window=10
Corpus: TaggedDocument(['C-Eb7-G#-B7', 'Eb7-G#-B7-E', 'G#-B7-E-G7', 'B7-E-G7-Gm', 'E-G7-Gm-C7', 'G7-Gm-C7-F', 'Gm-C7-F-G#7', 'C7-F-G#7-C#', 'F-G#7-C#-E7', 'G#7-C#-E7-Am', 'C#-E7-Am-D7', 'E7-Am-D7-Dm', 'Am-D7-Dm-G7'], [0, 0])


2021-11-17 14:38:05,722 : INFO : deleting the raw counts dictionary of 14361 items
2021-11-17 14:38:05,725 : INFO : sample=0 downsamples 0 most-common words
2021-11-17 14:38:05,727 : INFO : Doc2Vec lifecycle event {'msg': 'downsampling leaves estimated 66584 word corpus (100.0%% of prior 66584)', 'datetime': '2021-11-17T14:38:05.727348', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2021-11-17 14:38:05,929 : INFO : estimated required memory for 14361 words and 100 dimensions: 21911100 bytes
2021-11-17 14:38:05,933 : INFO : resetting layer weights
2021-11-17 14:38:05,954 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 2 workers on 14361 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=10 window=5 shrink_windows=True', 'datetime': '2021-11-17T14:38:05.954012', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GC

In [70]:
from gensim.models import KeyedVectors

from os import listdir
from os.path import isfile, join

names = [f for f in listdir(model_path) if isfile(join(model_path, f))]
print(names)
models = {}


for name in names:
    file_name = f"{model_path}/{name}"
    print(file_name)
    if not "npy" in name: ## !!! concat_x_y.model.syn1neg.npy cannot be loaded
      models[name] = Doc2Vec.load(file_name)

2021-11-17 15:04:16,271 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_concat_3_4.model
2021-11-17 15:04:16,341 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_concat_3_4.model.dv.* with mmap=None
2021-11-17 15:04:16,343 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_concat_3_4.model.wv.* with mmap=None
2021-11-17 15:04:16,345 : INFO : setting ignored attribute cum_table to None
2021-11-17 15:04:16,425 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_concat_3_4.model', 'datetime': '2021-11-17T15:04:16.425522', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-17 15:04:16,428 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_4_3.model
2021-11-17 15:04:16,461 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_4_3.model.dv.* with mmap=None
2021-11-17 15

['autos_d2v_concat_3_4.model', 'autos_d2v_pv-dbow_4_3.model', 'autos_d2v_pv-dbow_2_4.model', 'autos_d2v_pv-dbow_1_4.model', 'autos_d2v_pv-dm_1_4.model', 'autos_d2v_pv-dbow_1_3.model', 'autos_d2v_pv-dbow_3_3.model', 'autos_d2v_concat_1_10.model', 'autos_d2v_pv-dm_4_10.model', 'autos_d2v_pv-dm_3_4.model', 'autos_d2v_pv-dm_4_3.model', 'autos_d2v_pv-dm_3_3.model', 'autos_d2v_concat_4_10.model.syn1neg.npy', 'autos_d2v_pv-dm_4_2.model', 'autos_d2v_pv-dbow_2_10.model', 'autos_d2v_concat_4_2.model', 'autos_d2v_concat_4_4.model.syn1neg.npy', 'autos_d2v_pv-dbow_1_10.model', 'autos_d2v_pv-dbow_4_4.model', 'autos_d2v_concat_1_3.model', 'autos_d2v_concat_2_3.model', 'autos_d2v_concat_3_3.model', 'autos_d2v_concat_2_4.model', 'autos_d2v_pv-dm_3_2.model', 'autos_d2v_pv-dm_1_2.model', 'autos_d2v_pv-dbow_3_10.model', 'autos_d2v_concat_4_2.model.syn1neg.npy', 'autos_d2v_concat_4_4.model', 'autos_d2v_pv-dbow_4_10.model', 'autos_d2v_pv-dbow_3_2.model', 'autos_d2v_concat_1_2.model', 'autos_d2v_concat_1_4.m

2021-11-17 15:04:16,638 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_4_3.model', 'datetime': '2021-11-17T15:04:16.638271', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-17 15:04:16,639 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_2_4.model
2021-11-17 15:04:16,646 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_2_4.model.dv.* with mmap=None
2021-11-17 15:04:16,649 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_2_4.model.wv.* with mmap=None
2021-11-17 15:04:16,652 : INFO : setting ignored attribute cum_table to None
2021-11-17 15:04:16,673 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_2_4.model', 'datetime': '2021-11-17T15:04:16.673910', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]'

./models_doc2vec/autos_d2v_pv-dbow_2_4.model
./models_doc2vec/autos_d2v_pv-dbow_1_4.model
./models_doc2vec/autos_d2v_pv-dm_1_4.model
./models_doc2vec/autos_d2v_pv-dbow_1_3.model
./models_doc2vec/autos_d2v_pv-dbow_3_3.model
./models_doc2vec/autos_d2v_concat_1_10.model


2021-11-17 15:04:16,846 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_concat_1_10.model.dv.* with mmap=None
2021-11-17 15:04:16,848 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_concat_1_10.model.wv.* with mmap=None
2021-11-17 15:04:16,851 : INFO : setting ignored attribute cum_table to None
2021-11-17 15:04:16,858 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_concat_1_10.model', 'datetime': '2021-11-17T15:04:16.858363', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-17 15:04:16,861 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dm_4_10.model
2021-11-17 15:04:16,888 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dm_4_10.model.dv.* with mmap=None
2021-11-17 15:04:16,892 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dm_4_10.model.wv.* with mm

...

./models_doc2vec/autos_d2v_pv-dm_3_10.model


2021-11-17 15:04:21,986 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dm_3_10.model.dv.* with mmap=None
2021-11-17 15:04:21,989 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dm_3_10.model.wv.* with mmap=None
2021-11-17 15:04:21,992 : INFO : setting ignored attribute cum_table to None
2021-11-17 15:04:22,079 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dm_3_10.model', 'datetime': '2021-11-17T15:04:22.079776', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-17 15:04:22,082 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_concat_2_10.model
2021-11-17 15:04:22,108 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_concat_2_10.model.dv.* with mmap=None
2021-11-17 15:04:22,110 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_concat_2_10.model.wv.* with mm

./models_doc2vec/autos_d2v_concat_2_10.model
./models_doc2vec/autos_d2v_pv-dm_2_4.model
./models_doc2vec/autos_d2v_concat_4_10.model


2021-11-17 15:04:22,433 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_concat_4_10.model', 'datetime': '2021-11-17T15:04:22.433550', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-17 15:04:22,435 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_3_4.model
2021-11-17 15:04:22,455 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_4.model.dv.* with mmap=None
2021-11-17 15:04:22,456 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_4.model.wv.* with mmap=None
2021-11-17 15:04:22,457 : INFO : setting ignored attribute cum_table to None
2021-11-17 15:04:22,550 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_3_4.model', 'datetime': '2021-11-17T15:04:22.550273', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]'

./models_doc2vec/autos_d2v_concat_4_3.model.syn1neg.npy
./models_doc2vec/autos_d2v_pv-dbow_3_4.model


In [71]:
names = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print(names)
data = {}

for name in names:
    file_name = f"{data_path}/{name}"
    print(file_name)
    with open(file_name, 'rb') as f:
        data[name] = pickle.load(f)

['autos_d2v_pv-dm_1_4.train', 'autos_d2v_concat_3_4.train', 'autos_d2v_concat_4_4.train', 'autos_d2v_pv-dm_1_3.train', 'autos_d2v_concat_4_3.train', 'autos_d2v_concat_3_10.train', 'autos_d2v_pv-dbow_3_4.train', 'autos_d2v_pv-dm_1_2.train', 'autos_d2v_pv-dbow_3_2.train', 'autos_d2v_pv-dbow_3_10.train', 'autos_d2v_pv-dbow_4_3.train', 'autos_d2v_pv-dm_3_10.train', 'autos_d2v_concat_2_10.train', 'autos_d2v_pv-dbow_2_2.train', 'autos_d2v_pv-dm_2_4.train', 'autos_d2v_concat_1_4.train', 'autos_d2v_pv-dbow_1_10.train', 'autos_d2v_concat_2_4.train', 'autos_d2v_concat_2_3.train', 'autos_d2v_pv-dbow_2_3.train', 'autos_d2v_concat_4_10.train', 'autos_d2v_pv-dbow_2_4.train', 'autos_d2v_pv-dbow_1_4.train', 'autos_d2v_pv-dm_3_3.train', 'autos_d2v_pv-dm_2_10.train', 'autos_d2v_pv-dm_1_10.train', 'autos_d2v_pv-dm_4_2.train', 'autos_d2v_pv-dbow_4_2.train', 'autos_d2v_pv-dm_3_4.train', 'autos_d2v_pv-dbow_3_3.train', 'autos_d2v_pv-dm_3_2.train', 'autos_d2v_concat_4_2.train', 'autos_d2v_pv-dm_4_4.train', 'a

### Assess the best Model

In [75]:
#best_model = 'autos_d2v_pv-dbow_3_4'
best_model = 'autos_d2v_concat_1_3'

In [76]:
model = models[f"{best_model}.model"]

#### Determine Similarity of Tunes

In [78]:
assert(len(model.dv.vectors == len(list(title_to_sectionid.keys()))))

In [81]:
tunes_eval_list = [
                   'Sweet Sue, Just You',
  'These Foolish Things', 
  'Blue Moon',
  'Hundred Years From Today, A',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose',
  'Misty'
]

tunes_eval_list = [
  'These Foolish Things', 
]

In [94]:
df_sim = pd.DataFrame(columns=['title1', 'model', 'sim_section', 'sim_value'])
for tune in tunes_eval_list:
  print(tune)
  for section in title_to_sectionid[tune]:
      print(f"\n*** {sectionid_to_section[section]} ***")
      for model_name, model in models.items():
          print(f"{model_name}")
          sim = model.dv.most_similar(section)
          #print(f"'{sectionid_to_section[section]}' is most similar to:")
          for index, value in sim:
              print(f'\t{value:.3f}: {sectionid_to_section[index]}')
              df_sim.loc[len(df_sim)] = [tune, model_name, sectionid_to_section[section], f"{value:.3f}"]



These Foolish Things

*** These Foolish Things, section1 (A) ***
autos_d2v_concat_3_4.model
	0.334: Red Sails In The Sunset, section1 (A)
	0.310: Birth Of The Blues, The, section1 (A)
	0.310: Sorcerer, The, section0 (nan)
	0.300: Rosetta, section2 (A)
	0.295: How Deep Is The Ocean, section4 (C)
	0.294: Moontrane, The, section3 (B)
	0.294: Blue Moon, section4 (A)
	0.294: What A Little Moonlight Can Do, section3 (C)
	0.288: South Of The Border, section2 (A)
	0.288: Honeysuckle Rose, section4 (A)
autos_d2v_pv-dbow_4_3.model
	0.933: I'm Old Fashioned, section3 (C)
	0.932: These Foolish Things, section2 (A)
	0.916: These Foolish Things, section4 (A)
	0.891: Blue Room, The, section1 (A)
	0.884: Way You Look Tonight, The, section1 (A)
	0.881: Blue Room, The, section2 (A)
	0.876: Emily, section2 (B)
	0.876: Blue Room, The, section4 (C)
	0.874: Smoke Gets In Your Eyes, section4 (A)
	0.869: Moonlight In Vermont, section2 (A)
autos_d2v_pv-dbow_2_4.model
	0.909: These Foolish Things, section4 (A)


In [95]:
df_sim

,title1,model,sim_section,sim_value
0,These Foolish Things,autos_d2v_concat_3_4.model,"These Foolish Things, section1 (A)",value:.3f
1,These Foolish Things,autos_d2v_concat_3_4.model,"These Foolish Things, section1 (A)",value:.3f
2,These Foolish Things,autos_d2v_concat_3_4.model,"These Foolish Things, section1 (A)",value:.3f
3,These Foolish Things,autos_d2v_concat_3_4.model,"These Foolish Things, section1 (A)",value:.3f
4,These Foolish Things,autos_d2v_concat_3_4.model,"These Foolish Things, section1 (A)",value:.3f
...,...,...,...,...
1915,These Foolish Things,autos_d2v_pv-dbow_3_4.model,"These Foolish Things, section4 (A)",value:.3f
1916,These Foolish Things,autos_d2v_pv-dbow_3_4.model,"These Foolish Things, section4 (A)",value:.3f
1917,These Foolish Things,autos_d2v_pv-dbow_3_4.model,"These Foolish Things, section4 (A)",value:.3f
1918,These Foolish Things,autos_d2v_pv-dbow_3_4.model,"These Foolish Things, section4 (A)",value:.3f


#### Evaluate Similarity for Jazz Contrafacts



In [ ]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2', 'doc2vec']
df_eval_doc2vec = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
  row = [index_to_title[tune1], 
         index_to_title[tune2]]
  row.append(model.dv.similarity(tune1, tune2))
  
  df_eval_doc2vec.loc[len(df_eval_doc2vec)] = row

df_eval_doc2vec

,Tune1,Tune2,doc2vec
0,26-2,Confirmation,0.140248
1,52nd Street Theme,I Got Rhythm,0.617661
2,Ablution,All The Things You Are,0.880569
3,Anthropology,I Got Rhythm,0.245398
4,Ballade,As Long As I Live,0.008885
5,Bright Mississippi,Sweet Georgia Brown,0.699102
6,C.T.A.,I Got Rhythm,0.011225
7,Celia,I Got Rhythm,-0.261150
8,Cottontail,I Got Rhythm,0.727402
9,Countdown,Tune Up,0.533785


##### Compare results from TF-IDF and doc2vec



In [ ]:
df_eval_tfidf.merge(df_eval_doc2vec)

,Tune1,Tune2,cosval_ngram1,cosval_ngram2,cosval_ngram3,cosval_ngram4,doc2vec
0,26-2,Confirmation,0.568533,0.564252,0.574305,0.577075,0.140248
1,52nd Street Theme,I Got Rhythm,0.848234,0.848234,0.841295,0.833057,0.617661
2,Ablution,All The Things You Are,0.886735,0.877070,0.879810,0.878762,0.880569
3,Anthropology,I Got Rhythm,0.758506,0.758506,0.766709,0.764410,0.245398
4,Ballade,As Long As I Live,0.634559,0.646146,0.637085,0.628598,0.008885
5,Bright Mississippi,Sweet Georgia Brown,0.889316,0.884625,0.874615,0.862008,0.699102
6,C.T.A.,I Got Rhythm,0.624415,0.625980,0.635572,0.640375,0.011225
7,Celia,I Got Rhythm,0.502398,0.486288,0.484689,0.471808,-0.261150
8,Cottontail,I Got Rhythm,0.913729,0.913729,0.909956,0.905746,0.727402
9,Countdown,Tune Up,0.744414,0.725241,0.703373,0.691400,0.533785


##### Verbose Testing Output

In [ ]:
similarity_found = 0

for tune1, tune2 in contrafacts.items():
  index1 = title_to_index[tune1]
  index2 = title_to_index[tune2]

  sim = model.dv.similarity(index1, index2)
  most_sim = model.dv.most_similar(index1, topn=5)
  
  # check whether the index2 was found among the topn similar tunes
  for i in most_sim:
    if index2 == i[0]:
      similarity_found += 1

  print()
  print(f"'{tune1}' to '{tune2}': {sim}")
  print(f"'{tune1}' is most similar to:")
  for index, value in most_sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')


print()
num_tunes = len(contrafacts)
print(f"Number of tunes tested: {num_tunes}")
print(f"Number of similarities found: {similarity_found}")
print(f"Result: {100 * similarity_found / num_tunes}%")


'26-2' to 'Confirmation': 0.14024804532527924
'26-2' is most similar to:
	0.730: Whisper Not
	0.714: Bossa Antigua
	0.699: Drop Me Off In Harlem
	0.683: You Can Depend On Me
	0.681: Perhaps

'52nd Street Theme' to 'I Got Rhythm': 0.6176611185073853
'52nd Street Theme' is most similar to:
	0.869: I've Got My Fingers Crossed
	0.867: When The Moon Comes Over The Mountain
	0.854: Mack The Knife
	0.837: Why Do I Love You
	0.820: Soon

'Ablution' to 'All The Things You Are': 0.8805689811706543
'Ablution' is most similar to:
	0.881: All The Things You Are
	0.756: Everybody's Song But My Own
	0.712: Zingaro (Retrato Em Branco E Preto)
	0.712: Days Of Wine And Roses
	0.684: One Note Samba

'Anthropology' to 'I Got Rhythm': 0.24539807438850403
'Anthropology' is most similar to:
	0.975: Moose The Mooche
	0.968: Shaw 'Nuff
	0.964: Salt Peanuts
	0.963: No Moe
	0.957: Oleo

'Ballade' to 'As Long As I Live': 0.008885488845407963
'Ballade' is most similar to:
	0.813: Got A Match
	0.813: For Minors On

In [ ]:
model.dv.similarity(0, 1)

0.45295945

#### Download Model

In [ ]:
from google.colab import files
files.download(f"{model_path}/{best_model}.model")
files.download(f"{data_path}/{best_model}.train")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>